In [1]:
!pip install nltk
!pip install textblob
!pip install wordcloud

from warnings import filterwarnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("vader_lexicon")

filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

[nltk_data] Downloading package stopwords to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df_reviews=pd.read_csv(r"reviews.csv")
df_reviews.head(10)

,Name,Stars,Title,Date,Description
0,Keshav agarwal,5.00,5.0 out of 5 stars\nWatch,17 June 2024,The watch is amazing and I liked it The watch ...
1,Mustaque Ali Ansari,4.00,4.0 out of 5 stars\nValue for money and a nice...,3 February 2023,Bought this piece for a very peculiar reason. ...
2,I bought This watch and it arrived at the same...,5.00,5.0 out of 5 stars\nBest watch ever...,10 April 2024,Bought this piece for a very peculiar reason. ...
3,Harsha n.,5.00,5.0 out of 5 stars\nBest in class ever...,13 May 2024,- The screen has 60 hz refresh rate that reall...
4,Prasadh Laagad,4.00,4.0 out of 5 stars\nLooks Good!,20 June 2024,"Rectangular shape, good looks, quality of prod..."
5,Bharathwaj Kuppan,5.00,5.0 out of 5 stars\nBest Smartwatch in this se...,10 June 2024,I ordered Noise Colorfit Pro Max 5 and it got ...
6,Subhendu,3.00,3.0 out of 5 stars\nGood product,21 June 2024,Good product
7,AmzonCust,4.00,4.0 out of 5 stars\nThis is a good watch with ...,2 January 2024,"This is a good looking all ""bells and whistles..."
8,hisham Rahman,4.00,"4.0 out of 5 stars\n""ColorFit Pro 5 Max"": It's...",12 June 2024,Product is good. Nothing much to complain.Howe...
9,Abhinav,4.00,4.0 out of 5 stars\nSenseless sensors,20 June 2024,When you will connect your smartwatch on the ...


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from textblob.np_extractors import ConllExtractor
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from nltk.classify import NaiveBayesClassifier
from cleantext import clean

In [4]:
df=pd.DataFrame(df_reviews['Description'])
df

,Description
0,The watch is amazing and I liked it The watch ...
1,Bought this piece for a very peculiar reason. ...
2,Bought this piece for a very peculiar reason. ...
3,- The screen has 60 hz refresh rate that reall...
4,"Rectangular shape, good looks, quality of prod..."
...,...
95,It’s good to looks wear
96,"The watch is good after 7 day review, nice int..."
97,NaN
98,"Nice watch, durable"


In [5]:
def text_preprocessing(dataframe, dependent_var):
  # Normalizing Case Folding - Uppercase to Lowercase
  dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

  # Removing Punctuation
  dataframe[dependent_var] = dataframe[dependent_var].str.replace('[^\w\s]','')

  # Removing Numbers
  dataframe[dependent_var] = dataframe[dependent_var].str.replace('\d','')

  # StopWords
  sw = stopwords.words('english')
  dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

  # Remove Rare Words
  temp_df = pd.Series(' '.join(dataframe[dependent_var]).split()).value_counts()
  drops = temp_df[temp_df <= 1]
  dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops))

  # Lemmatize
  dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

  return dataframe

In [6]:
df = text_preprocessing(df, "Description")

In [7]:
df["Description"].head()

0    watch amazing liked watch really loud enough s...
1    bought piece peculiar reason needed watch remi...
2    bought piece peculiar reason needed watch remi...
3    screen hz refresh rate really make responsive ...
4    good look quality product hardware seems good ...
Name: Description, dtype: object

In [8]:
import emoji

# Replace emojis with an empty string
df['Description'] = df['Description'].apply(lambda s: emoji.replace_emoji(str(s), '') if isinstance(s, str) else '')
df

,Description
0,watch amazing liked watch really loud enough s...
1,bought piece peculiar reason needed watch remi...
2,bought piece peculiar reason needed watch remi...
3,screen hz refresh rate really make responsive ...
4,good look quality product hardware seems good ...
...,...
95,good look
96,watch good day review nice interface backup da...
97,nan
98,nice watch


In [9]:
# Convert all strings in the 'Description' column to lowercase
df['Description'] = df['Description'].str.lower()
df

,Description
0,watch amazing liked watch really loud enough s...
1,bought piece peculiar reason needed watch remi...
2,bought piece peculiar reason needed watch remi...
3,screen hz refresh rate really make responsive ...
4,good look quality product hardware seems good ...
...,...
95,good look
96,watch good day review nice interface backup da...
97,nan
98,nice watch


In [10]:
def create_polarity_scores(dataframe, dependent_var):
  sia = SentimentIntensityAnalyzer()
  dataframe["polarity_score"] = dataframe[dependent_var].apply(lambda x: sia.polarity_scores(x)["compound"])

In [11]:
create_polarity_scores(df, "Description")

In [12]:
df.head()

,Description,polarity_score
0,watch amazing liked watch really loud enough s...,0.98
1,bought piece peculiar reason needed watch remi...,0.99
2,bought piece peculiar reason needed watch remi...,0.99
3,screen hz refresh rate really make responsive ...,0.99
4,good look quality product hardware seems good ...,0.83


# Feature Engineering

In [13]:
# Create Lables
def create_label(dataframe, dependent_var, independent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe[independent_var] = dataframe[dependent_var].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
    dataframe[independent_var] = LabelEncoder().fit_transform(dataframe[independent_var])

    X = dataframe[dependent_var]
    y = dataframe[independent_var]

    return X, y

In [14]:
X, y = create_label(df, "Description", "sentiment_label")

In [15]:
# Split Dataset
def split_dataset(dataframe, X, y):
    train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=1)
    return train_x, test_x, train_y, test_y

In [16]:
train_x, test_x, train_y, test_y = split_dataset(df, X, y)

In [17]:
def create_features_count(train_x, test_x):
    # Count Vectors
    vectorizer = CountVectorizer()
    x_train_count_vectorizer = vectorizer.fit_transform(train_x)
    x_test_count_vectorizer = vectorizer.fit_transform(test_x)

    return x_train_count_vectorizer, x_test_count_vectorizer

In [18]:
x_train_count_vectorizer, x_test_count_vectorizer = create_features_count(train_x, test_x)

In [19]:
def create_features_TFIDF_word(train_x, test_x):
    # TF-IDF word
    tf_idf_word_vectorizer = TfidfVectorizer()
    x_train_tf_idf_word = tf_idf_word_vectorizer.fit_transform(train_x)
    x_test_tf_idf_word = tf_idf_word_vectorizer.fit_transform(test_x)

    return x_train_tf_idf_word, x_test_tf_idf_word

In [20]:
x_train_tf_idf_word, x_test_tf_idf_word = create_features_TFIDF_word(train_x, test_x)

In [21]:
def create_features_TFIDF_ngram(train_x, test_x):
    # TF-IDF ngram
    tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2,3))
    x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.fit_transform(train_x)
    x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.fit_transform(test_x)

    return x_train_tf_idf_ngram, x_test_tf_idf_ngram

In [22]:
x_train_tf_idf_ngram, x_test_tf_idf_ngram = create_features_TFIDF_ngram(train_x, test_x)

In [23]:
def create_features_TFIDF_chars(train_x, test_x):
    # TF-IDF Characters
    tf_idf_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2,3))
    x_train_tf_idf_chars = tf_idf_chars_vectorizer.fit_transform(train_x)
    x_test_tf_idf_chars = tf_idf_chars_vectorizer.fit_transform(test_x)

    return x_train_tf_idf_chars, x_test_tf_idf_chars

In [24]:
x_train_tf_idf_chars, x_test_tf_idf_chars = create_features_TFIDF_chars(train_x, test_x)

In [25]:
# Logistic Regression
def crate_model_logistic(train_x, test_x):
  # Count
    x_train_count_vectorizer, x_test_count_vectorizer = create_features_count(train_x, test_x)
    loj_count = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_count = loj_count.fit(x_train_count_vectorizer, train_y)
    accuracy_count = cross_val_score(loj_model_count, x_test_count_vectorizer, test_y, cv=10).mean()
    print("Accuracy - Count Vectors: %.3f" % accuracy_count)

    # TF-IDF Word
    x_train_tf_idf_word, x_test_tf_idf_word = create_features_TFIDF_word(train_x, test_x)
    loj_word = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_word = loj_word.fit(x_train_tf_idf_word, train_y)
    accuracy_word = cross_val_score(loj_model_word, x_test_tf_idf_word, test_y, cv=10).mean()
    print("Accuracy - TF-IDF Word: %.3f" % accuracy_word)

    # TF-IDF ngram
    x_train_tf_idf_ngram, x_test_tf_idf_ngram = create_features_TFIDF_ngram(train_x, test_x)
    loj_ngram = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_ngram = loj_ngram.fit(x_train_tf_idf_ngram, train_y)
    accuracy_ngram = cross_val_score(loj_model_ngram, x_test_tf_idf_ngram, test_y, cv=10).mean()
    print("Accuracy TF-IDF ngram: %.3f" % accuracy_ngram)

    # TF-IDF chars

    loj_chars = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_chars = loj_chars.fit(x_train_tf_idf_chars, train_y)
    accuracy_chars = cross_val_score(loj_model_chars, x_test_tf_idf_chars, test_y, cv=10).mean()
    print("Accuracy TF-IDF Characters: %.3f" % accuracy_chars)

    return loj_model_count, loj_model_word, loj_model_ngram, loj_model_chars

In [26]:
loj_model_count, loj_model_word, loj_model_ngram, loj_model_chars = crate_model_logistic(train_x, test_x)

Accuracy - Count Vectors: 0.867
Accuracy - TF-IDF Word: 0.867
Accuracy TF-IDF ngram: 0.867
Accuracy TF-IDF Characters: 0.867


In [27]:
# Random Forest
def crate_model_randomforest(train_x, test_x):
    # Count
    x_train_count_vectorizer, x_test_count_vectorizer = create_features_count(train_x, test_x)
    rf_count = RandomForestClassifier()
    rf_model_count = rf_count.fit(x_train_count_vectorizer, train_y)
    accuracy_count = cross_val_score(rf_model_count, x_test_count_vectorizer, test_y, cv=10).mean()
    print("Accuracy - Count Vectors: %.3f" % accuracy_count)

    # TF-IDF Word
    x_train_tf_idf_word, x_test_tf_idf_word = create_features_TFIDF_word(train_x, test_x)
    rf_word = RandomForestClassifier()
    rf_model_word = rf_word.fit(x_train_tf_idf_word, train_y)
    accuracy_word = cross_val_score(rf_model_word, x_test_tf_idf_word, test_y, cv=10).mean()
    print("Accuracy - TF-IDF Word: %.3f" % accuracy_word)

    # TF-IDF ngram
    x_train_tf_idf_ngram, x_test_tf_idf_ngram = create_features_TFIDF_ngram(train_x, test_x)
    rf_ngram = RandomForestClassifier()
    rf_model_ngram = rf_ngram.fit(x_train_tf_idf_ngram, train_y)
    accuracy_ngram = cross_val_score(rf_model_ngram, x_test_tf_idf_ngram, test_y, cv=10).mean()
    print("Accuracy TF-IDF ngram: %.3f" % accuracy_ngram)

    # TF-IDF chars

    rf_chars = RandomForestClassifier()
    rf_model_chars = rf_chars.fit(x_train_tf_idf_chars, train_y)
    accuracy_chars = cross_val_score(rf_model_chars, x_test_tf_idf_chars, test_y, cv=10).mean()
    print("Accuracy TF-IDF Characters: %.3f" % accuracy_chars)

    return rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars

In [28]:
rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars = crate_model_randomforest(train_x, test_x)

Accuracy - Count Vectors: 0.883
Accuracy - TF-IDF Word: 0.800
Accuracy TF-IDF ngram: 0.267
Accuracy TF-IDF Characters: 0.883


# Model Tuning

In [29]:
def model_tuning_randomforest(train_x, test_x):
    # Count
    x_train_count_vectorizer, x_test_count_vectorizer = create_features_count(train_x, test_x)
    rf_model_count = RandomForestClassifier(random_state=1)
    rf_params = {"max_depth": [2,5,8, None],
                 "max_features": [2,5,8, "auto"],
                 "n_estimators": [100,500,1000],
                 "min_samples_split": [2,5,10]}
    rf_best_grid = GridSearchCV(rf_model_count, rf_params, cv=10, n_jobs=-1, verbose=False).fit(x_train_count_vectorizer, train_y)
    rf_model_count_final = rf_model_count.set_params(**rf_best_grid.best_params_, random_state=1).fit(x_train_count_vectorizer, train_y)
    accuracy_count = cross_val_score(rf_model_count_final, x_test_count_vectorizer, test_y, cv=10).mean()
    print("Accuracy - Count Vectors: %.3f" % accuracy_count)

    return rf_model_count_final

In [30]:
rf_model_count_final = model_tuning_randomforest(train_x, test_x)

Accuracy - Count Vectors: 0.867


# Prediction

In [31]:
def predict_count(train_x, model, new_comment):
    new_comment= pd.Series(new_comment)
    new_comment = CountVectorizer().fit(train_x).transform(new_comment)
    result = model.predict(new_comment)
    if result==1:
        print("Comment is Pozitive")
    else:
        print("Comment is Negative")

In [32]:
# Logistic Regression
predict_count(train_x, model=loj_model_count, new_comment="this product is very good :)")

Comment is Pozitive


In [33]:
# Random Forest
predict_count(train_x, model=rf_model_count, new_comment="this product is very bad :)")

Comment is Pozitive


In [34]:
# Sample Review
new_comment=pd.Series(df["Description"].sample(1).values)
new_comment

0    great product noise app experience also
dtype: object

In [35]:
# Sample Review
new_comment=pd.Series(df["Description"].sample(1).values)
new_comment

0    nice product
dtype: object

In [36]:
# Sample Review - Random Forest
predict_count(train_x, model=rf_model_count, new_comment=new_comment)

Comment is Pozitive


In [5]:
from warnings import filterwarnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
import emoji
import pickle

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("vader_lexicon")

filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

df_reviews = pd.read_csv(r"reviews.csv")
df = pd.DataFrame(df_reviews['Description'])

# Replace emojis with an empty string
df['Description'] = df['Description'].apply(lambda s: emoji.replace_emoji(str(s), '') if isinstance(s, str) else '')

def text_preprocessing(dataframe, dependent_var):
    # Normalizing Case Folding - Uppercase to Lowercase
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

    # Removing Punctuation
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('[^\w\s]', '', regex=True)

    # Removing Numbers
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('\d', '', regex=True)

    # StopWords
    sw = stopwords.words('english')
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

    # Remove Rare Words
    temp_df = pd.Series(' '.join(dataframe[dependent_var]).split()).value_counts()
    drops = temp_df[temp_df <= 1]
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops))

    # Lemmatize
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

    return dataframe

df = text_preprocessing(df, "Description")

def create_polarity_scores(dataframe, dependent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe["polarity_score"] = dataframe[dependent_var].apply(lambda x: sia.polarity_scores(x)["compound"])

create_polarity_scores(df, "Description")

def create_label(dataframe, dependent_var, independent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe[independent_var] = dataframe[dependent_var].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
    dataframe[independent_var] = LabelEncoder().fit_transform(dataframe[independent_var])

    X = dataframe[dependent_var]
    y = dataframe[independent_var]

    return X, y

X, y = create_label(df, "Description", "sentiment_label")

def split_dataset(dataframe, X, y):
    train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=1)
    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = split_dataset(df, X, y)

def create_features_count(train_x, test_x):
    vectorizer = CountVectorizer()
    x_train_count_vectorizer = vectorizer.fit_transform(train_x)
    x_test_count_vectorizer = vectorizer.transform(test_x)

    return x_train_count_vectorizer, x_test_count_vectorizer, vectorizer

x_train_count_vectorizer, x_test_count_vectorizer, count_vectorizer = create_features_count(train_x, test_x)

def create_features_TFIDF_word(train_x, test_x):
    tf_idf_word_vectorizer = TfidfVectorizer()
    x_train_tf_idf_word = tf_idf_word_vectorizer.fit_transform(train_x)
    x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

    return x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer

x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer = create_features_TFIDF_word(train_x, test_x)

def create_features_TFIDF_ngram(train_x, test_x):
    tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2, 3))
    x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.fit_transform(train_x)
    x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

    return x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer

x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer = create_features_TFIDF_ngram(train_x, test_x)

def create_features_TFIDF_chars(train_x, test_x):
    tf_idf_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 3))
    x_train_tf_idf_chars = tf_idf_chars_vectorizer.fit_transform(train_x)
    x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

    return x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer

x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer = create_features_TFIDF_chars(train_x, test_x)

# Logistic Regression
def create_model_logistic(train_x, test_x):
    # Count
    x_train_count_vectorizer, x_test_count_vectorizer, count_vectorizer = create_features_count(train_x, test_x)
    loj_count = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_count = loj_count.fit(x_train_count_vectorizer, train_y)
    accuracy_count = cross_val_score(loj_model_count, x_test_count_vectorizer, test_y, cv=10).mean()
    print("Accuracy - Count Vectors: %.3f" % accuracy_count)

    # TF-IDF Word
    x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer = create_features_TFIDF_word(train_x, test_x)
    loj_word = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_word = loj_word.fit(x_train_tf_idf_word, train_y)
    accuracy_word = cross_val_score(loj_model_word, x_test_tf_idf_word, test_y, cv=10).mean()
    print("Accuracy - TF-IDF Word: %.3f" % accuracy_word)

    # TF-IDF ngram
    x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer = create_features_TFIDF_ngram(train_x, test_x)
    loj_ngram = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_ngram = loj_ngram.fit(x_train_tf_idf_ngram, train_y)
    accuracy_ngram = cross_val_score(loj_model_ngram, x_test_tf_idf_ngram, test_y, cv=10).mean()
    print("Accuracy TF-IDF ngram: %.3f" % accuracy_ngram)

    # TF-IDF chars
    x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer = create_features_TFIDF_chars(train_x, test_x)
    loj_chars = LogisticRegression(solver='lbfgs', max_iter=1000)
    loj_model_chars = loj_chars.fit(x_train_tf_idf_chars, train_y)
    accuracy_chars = cross_val_score(loj_model_chars, x_test_tf_idf_chars, test_y, cv=10).mean()
    print("Accuracy TF-IDF Characters: %.3f" % accuracy_chars)

    return loj_model_count, loj_model_word, loj_model_ngram, loj_model_chars
    
loj_model_count, loj_model_word, loj_model_ngram, loj_model_chars = create_model_logistic(train_x, test_x)
    
def predict_count(model, vectorizer, new_comment):
    new_comment = pd.Series(new_comment)
    new_comment = vectorizer.transform(new_comment)
    result = model.predict(new_comment)
    if result == 1:
        print("Comment is Positive")
    else:
        print("Comment is Negative")

# Example usage
predict_count(model=loj_model_count, vectorizer=count_vectorizer, new_comment="this product is very good :)")

# Save the models to separate pickle files
with open('model_count.pkl', 'wb') as file:
    pickle.dump(loj_model_count, file)
print("Count Vectorizer model saved successfully")

with open('model_word.pkl', 'wb') as file:
    pickle.dump(loj_model_word, file)
print("TF-IDF Word model saved successfully")

with open('model_ngram.pkl', 'wb') as file:
    pickle.dump(loj_model_ngram, file)
print("TF-IDF Ngram model saved successfully")

with open('model_chars.pkl', 'wb') as file:
    pickle.dump(loj_model_chars, file)
print("TF-IDF Chars model saved successfully")


[nltk_data] Downloading package stopwords to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Accuracy - Count Vectors: 0.867
Accuracy - TF-IDF Word: 0.867
Accuracy TF-IDF ngram: 0.867
Accuracy TF-IDF Characters: 0.867
Comment is Positive
Count Vectorizer model saved successfully
TF-IDF Word model saved successfully
TF-IDF Ngram model saved successfully
TF-IDF Chars model saved successfully


In [2]:
from warnings import filterwarnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
import emoji
import pickle

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("vader_lexicon")

filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

df_reviews = pd.read_csv(r"reviews.csv")
df = pd.DataFrame(df_reviews['Description'])

# Replace emojis with an empty string
df['Description'] = df['Description'].apply(lambda s: emoji.replace_emoji(str(s), '') if isinstance(s, str) else '')

def text_preprocessing(dataframe, dependent_var):
    # Normalizing Case Folding - Uppercase to Lowercase
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

    # Removing Punctuation
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('[^\w\s]', '', regex=True)

    # Removing Numbers
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('\d', '', regex=True)

    # StopWords
    sw = stopwords.words('english')
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

    # Remove Rare Words
    temp_df = pd.Series(' '.join(dataframe[dependent_var]).split()).value_counts()
    drops = temp_df[temp_df <= 1]
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops))

    # Lemmatize
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

    return dataframe

df = text_preprocessing(df, "Description")

def create_polarity_scores(dataframe, dependent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe["polarity_score"] = dataframe[dependent_var].apply(lambda x: sia.polarity_scores(x)["compound"])

create_polarity_scores(df, "Description")

def create_label(dataframe, dependent_var, independent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe[independent_var] = dataframe[dependent_var].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
    dataframe[independent_var] = LabelEncoder().fit_transform(dataframe[independent_var])

    X = dataframe[dependent_var]
    y = dataframe[independent_var]

    return X, y

X, y = create_label(df, "Description", "sentiment_label")

def split_dataset(dataframe, X, y):
    train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=1)
    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = split_dataset(df, X, y)

def create_features_count(train_x, test_x):
    vectorizer = CountVectorizer()
    x_train_count_vectorizer = vectorizer.fit_transform(train_x)
    x_test_count_vectorizer = vectorizer.transform(test_x)

    return x_train_count_vectorizer, x_test_count_vectorizer, vectorizer

x_train_count_vectorizer, x_test_count_vectorizer, count_vectorizer = create_features_count(train_x, test_x)

def create_features_TFIDF_word(train_x, test_x):
    tf_idf_word_vectorizer = TfidfVectorizer()
    x_train_tf_idf_word = tf_idf_word_vectorizer.fit_transform(train_x)
    x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

    return x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer

x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer = create_features_TFIDF_word(train_x, test_x)

def create_features_TFIDF_ngram(train_x, test_x):
    tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2, 3))
    x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.fit_transform(train_x)
    x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

    return x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer

x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer = create_features_TFIDF_ngram(train_x, test_x)

def create_features_TFIDF_chars(train_x, test_x):
    tf_idf_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 3))
    x_train_tf_idf_chars = tf_idf_chars_vectorizer.fit_transform(train_x)
    x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

    return x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer

x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer = create_features_TFIDF_chars(train_x, test_x)

# Random Forest
def crate_model_randomforest(train_x, test_x):
    # Count
    x_train_count_vectorizer, x_test_count_vectorizer = create_features_count(train_x, test_x)
    rf_count = RandomForestClassifier()
    rf_model_count = rf_count.fit(x_train_count_vectorizer, train_y)
    accuracy_count = cross_val_score(rf_model_count, x_test_count_vectorizer, test_y, cv=10).mean()
    print("Accuracy - Count Vectors: %.3f" % accuracy_count)

    # TF-IDF Word
    x_train_tf_idf_word, x_test_tf_idf_word = create_features_TFIDF_word(train_x, test_x)
    rf_word = RandomForestClassifier()
    rf_model_word = rf_word.fit(x_train_tf_idf_word, train_y)
    accuracy_word = cross_val_score(rf_model_word, x_test_tf_idf_word, test_y, cv=10).mean()
    print("Accuracy - TF-IDF Word: %.3f" % accuracy_word)

    # TF-IDF ngram
    x_train_tf_idf_ngram, x_test_tf_idf_ngram = create_features_TFIDF_ngram(train_x, test_x)
    rf_ngram = RandomForestClassifier()
    rf_model_ngram = rf_ngram.fit(x_train_tf_idf_ngram, train_y)
    accuracy_ngram = cross_val_score(rf_model_ngram, x_test_tf_idf_ngram, test_y, cv=10).mean()
    print("Accuracy TF-IDF ngram: %.3f" % accuracy_ngram)

    # TF-IDF chars

    rf_chars = RandomForestClassifier()
    rf_model_chars = rf_chars.fit(x_train_tf_idf_chars, train_y)
    accuracy_chars = cross_val_score(rf_model_chars, x_test_tf_idf_chars, test_y, cv=10).mean()
    print("Accuracy TF-IDF Characters: %.3f" % accuracy_chars)

    return rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars

rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars = crate_model_randomforest(train_x, test_x)

    
def predict_count(model, vectorizer, new_comment):
    new_comment = pd.Series(new_comment)
    new_comment = vectorizer.transform(new_comment)
    result = model.predict(new_comment)
    if result == 1:
        print("Comment is Positive")
    else:
        print("Comment is Negative")

# Example usage
predict_count(model=loj_model_count, vectorizer=count_vectorizer, new_comment="this product is very good :)")

# Save the models to separate pickle files
with open('model_count.pkl', 'wb') as file:
    pickle.dump(loj_model_count, file)
print("Count Vectorizer model saved successfully")

with open('model_word.pkl', 'wb') as file:
    pickle.dump(loj_model_word, file)
print("TF-IDF Word model saved successfully")

with open('model_ngram.pkl', 'wb') as file:
    pickle.dump(loj_model_ngram, file)
print("TF-IDF Ngram model saved successfully")

with open('model_chars.pkl', 'wb') as file:
    pickle.dump(loj_model_chars, file)
print("TF-IDF Chars model saved successfully")


[nltk_data] Downloading package stopwords to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


ValueError: too many values to unpack (expected 2)

In [7]:
from warnings import filterwarnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from textblob import Word
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
import emoji
import pickle

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("vader_lexicon")

filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

# Load data
df_reviews = pd.read_csv(r"reviews.csv")
df = pd.DataFrame(df_reviews['Description'])

# Replace emojis with an empty string
df['Description'] = df['Description'].apply(lambda s: emoji.replace_emoji(str(s), '') if isinstance(s, str) else '')

def text_preprocessing(dataframe, dependent_var):
    # Normalizing Case Folding - Uppercase to Lowercase
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

    # Removing Punctuation
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('[^\w\s]', '', regex=True)

    # Removing Numbers
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('\d', '', regex=True)

    # StopWords
    sw = stopwords.words('english')
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

    # Remove Rare Words
    temp_df = pd.Series(' '.join(dataframe[dependent_var]).split()).value_counts()
    drops = temp_df[temp_df <= 1]
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops))

    # Lemmatize
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

    return dataframe

df = text_preprocessing(df, "Description")

def create_polarity_scores(dataframe, dependent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe["polarity_score"] = dataframe[dependent_var].apply(lambda x: sia.polarity_scores(x)["compound"])

create_polarity_scores(df, "Description")

def create_label(dataframe, dependent_var, independent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe[independent_var] = dataframe[dependent_var].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
    dataframe[independent_var] = LabelEncoder().fit_transform(dataframe[independent_var])

    X = dataframe[dependent_var]
    y = dataframe[independent_var]

    return X, y

X, y = create_label(df, "Description", "sentiment_label")

def split_dataset(dataframe, X, y):
    train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=1)
    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = split_dataset(df, X, y)

def create_features_count(train_x, test_x):
    vectorizer = CountVectorizer()
    x_train_count_vectorizer = vectorizer.fit_transform(train_x)
    x_test_count_vectorizer = vectorizer.transform(test_x)

    return x_train_count_vectorizer, x_test_count_vectorizer, vectorizer

x_train_count_vectorizer, x_test_count_vectorizer, count_vectorizer = create_features_count(train_x, test_x)

def create_features_TFIDF_word(train_x, test_x):
    tf_idf_word_vectorizer = TfidfVectorizer()
    x_train_tf_idf_word = tf_idf_word_vectorizer.fit_transform(train_x)
    x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

    return x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer

x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer = create_features_TFIDF_word(train_x, test_x)

def create_features_TFIDF_ngram(train_x, test_x):
    tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2, 3))
    x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.fit_transform(train_x)
    x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

    return x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer

x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer = create_features_TFIDF_ngram(train_x, test_x)

def create_features_TFIDF_chars(train_x, test_x):
    tf_idf_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 3))
    x_train_tf_idf_chars = tf_idf_chars_vectorizer.fit_transform(train_x)
    x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

    return x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer

x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer = create_features_TFIDF_chars(train_x, test_x)

# Random Forest
def create_model_randomforest(train_x, test_x):
    # Count
    rf_count = RandomForestClassifier()
    rf_model_count = rf_count.fit(x_train_count_vectorizer, train_y)
    accuracy_count = cross_val_score(rf_model_count, x_test_count_vectorizer, test_y, cv=10).mean()
    print("Accuracy - Count Vectors: %.3f" % accuracy_count)

    # TF-IDF Word
    rf_word = RandomForestClassifier()
    rf_model_word = rf_word.fit(x_train_tf_idf_word, train_y)
    accuracy_word = cross_val_score(rf_model_word, x_test_tf_idf_word, test_y, cv=10).mean()
    print("Accuracy - TF-IDF Word: %.3f" % accuracy_word)

    # TF-IDF ngram
    rf_ngram = RandomForestClassifier()
    rf_model_ngram = rf_ngram.fit(x_train_tf_idf_ngram, train_y)
    accuracy_ngram = cross_val_score(rf_model_ngram, x_test_tf_idf_ngram, test_y, cv=10).mean()
    print("Accuracy TF-IDF ngram: %.3f" % accuracy_ngram)

    # TF-IDF chars
    rf_chars = RandomForestClassifier()
    rf_model_chars = rf_chars.fit(x_train_tf_idf_chars, train_y)
    accuracy_chars = cross_val_score(rf_model_chars, x_test_tf_idf_chars, test_y, cv=10).mean()
    print("Accuracy TF-IDF Characters: %.3f" % accuracy_chars)

    return rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars

rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars = create_model_randomforest(train_x, test_x)

def predict_sentiment(model, vectorizer, new_comment):
    new_comment_series = pd.Series(new_comment)
    new_comment_transformed = vectorizer.transform(new_comment_series)
    result = model.predict(new_comment_transformed)
    if result == 1:
        print("Comment is Positive")
    else:
        print("Comment is Negative")

# Example usage
predict_sentiment(model=rf_model_count, vectorizer=count_vectorizer, new_comment="this product is very good :)")

# Save the models to separate pickle files
with open('rf_model_count.pkl', 'wb') as file_1:
    pickle.dump(rf_model_count, file_1)
print("Count Vectorizer model saved successfully")

with open('rf_model_word.pkl', 'wb') as file_2:
    pickle.dump(rf_model_word, file_2)
print("TF-IDF Word model saved successfully")

with open('rf_model_ngram.pkl', 'wb') as file_3:
    pickle.dump(rf_model_ngram, file_3)
print("TF-IDF Ngram model saved successfully")

with open('rf_model_chars.pkl', 'wb') as file_4:
    pickle.dump(rf_model_chars, file)
print("TF-IDF Chars model saved successfully")

with open('count_vectorizer.pkl', 'wb') as vec_1:
    pickle.dump(count_vectorizer, vec_1)
print("Count Vectorizer saved successfully")

with open('tfidf_word_vectorizer.pkl', 'wb') as vec_2:
    pickle.dump(tf_idf_word_vectorizer, vec_2)
print("TF-IDF Word Vectorizer saved successfully")

with open('tfidf_ngram_vectorizer.pkl', 'wb') as vec_3:
    pickle.dump(tf_idf_ngram_vectorizer, vec_3)
print("TF-IDF Ngram Vectorizer saved successfully")

with open('tfidf_chars_vectorizer.pkl', 'wb') as vec_4:
    pickle.dump(tf_idf_chars_vectorizer, vec_4)
print("TF-IDF Chars Vectorizer saved successfully")


[nltk_data] Downloading package stopwords to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Accuracy - Count Vectors: 0.917
Accuracy - TF-IDF Word: 0.900
Accuracy TF-IDF ngram: 0.867
Accuracy TF-IDF Characters: 0.850
Comment is Positive
Count Vectorizer model saved successfully
TF-IDF Word model saved successfully
TF-IDF Ngram model saved successfully


ValueError: write to closed file

In [8]:
import pickle
from warnings import filterwarnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from textblob import Word
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
import emoji
import pickle

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("vader_lexicon")

filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

# Load data
df_reviews = pd.read_csv(r"reviews.csv")
df = pd.DataFrame(df_reviews['Description'])

# Replace emojis with an empty string
df['Description'] = df['Description'].apply(lambda s: emoji.replace_emoji(str(s), '') if isinstance(s, str) else '')

def text_preprocessing(dataframe, dependent_var):
    # Normalizing Case Folding - Uppercase to Lowercase
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

    # Removing Punctuation
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('[^\w\s]', '', regex=True)

    # Removing Numbers
    dataframe[dependent_var] = dataframe[dependent_var].str.replace('\d', '', regex=True)

    # StopWords
    sw = stopwords.words('english')
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

    # Remove Rare Words
    temp_df = pd.Series(' '.join(dataframe[dependent_var]).split()).value_counts()
    drops = temp_df[temp_df <= 1]
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops))

    # Lemmatize
    dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

    return dataframe

df = text_preprocessing(df, "Description")

def create_polarity_scores(dataframe, dependent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe["polarity_score"] = dataframe[dependent_var].apply(lambda x: sia.polarity_scores(x)["compound"])

create_polarity_scores(df, "Description")

def create_label(dataframe, dependent_var, independent_var):
    sia = SentimentIntensityAnalyzer()
    dataframe[independent_var] = dataframe[dependent_var].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
    dataframe[independent_var] = LabelEncoder().fit_transform(dataframe[independent_var])

    X = dataframe[dependent_var]
    y = dataframe[independent_var]

    return X, y

X, y = create_label(df, "Description", "sentiment_label")

def split_dataset(dataframe, X, y):
    train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=1)
    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = split_dataset(df, X, y)

def create_features_count(train_x, test_x):
    vectorizer = CountVectorizer()
    x_train_count_vectorizer = vectorizer.fit_transform(train_x)
    x_test_count_vectorizer = vectorizer.transform(test_x)

    return x_train_count_vectorizer, x_test_count_vectorizer, vectorizer

x_train_count_vectorizer, x_test_count_vectorizer, count_vectorizer = create_features_count(train_x, test_x)

def create_features_TFIDF_word(train_x, test_x):
    tf_idf_word_vectorizer = TfidfVectorizer()
    x_train_tf_idf_word = tf_idf_word_vectorizer.fit_transform(train_x)
    x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

    return x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer

x_train_tf_idf_word, x_test_tf_idf_word, tf_idf_word_vectorizer = create_features_TFIDF_word(train_x, test_x)

def create_features_TFIDF_ngram(train_x, test_x):
    tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2, 3))
    x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.fit_transform(train_x)
    x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

    return x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer

x_train_tf_idf_ngram, x_test_tf_idf_ngram, tf_idf_ngram_vectorizer = create_features_TFIDF_ngram(train_x, test_x)

def create_features_TFIDF_chars(train_x, test_x):
    tf_idf_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 3))
    x_train_tf_idf_chars = tf_idf_chars_vectorizer.fit_transform(train_x)
    x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

    return x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer

x_train_tf_idf_chars, x_test_tf_idf_chars, tf_idf_chars_vectorizer = create_features_TFIDF_chars(train_x, test_x)

# Random Forest
def create_model_randomforest(train_x, test_x):
    # Count
    rf_count = RandomForestClassifier()
    rf_model_count = rf_count.fit(x_train_count_vectorizer, train_y)
    accuracy_count = cross_val_score(rf_model_count, x_test_count_vectorizer, test_y, cv=10).mean()
    print("Accuracy - Count Vectors: %.3f" % accuracy_count)

    # TF-IDF Word
    rf_word = RandomForestClassifier()
    rf_model_word = rf_word.fit(x_train_tf_idf_word, train_y)
    accuracy_word = cross_val_score(rf_model_word, x_test_tf_idf_word, test_y, cv=10).mean()
    print("Accuracy - TF-IDF Word: %.3f" % accuracy_word)

    # TF-IDF ngram
    rf_ngram = RandomForestClassifier()
    rf_model_ngram = rf_ngram.fit(x_train_tf_idf_ngram, train_y)
    accuracy_ngram = cross_val_score(rf_model_ngram, x_test_tf_idf_ngram, test_y, cv=10).mean()
    print("Accuracy TF-IDF ngram: %.3f" % accuracy_ngram)

    # TF-IDF chars
    rf_chars = RandomForestClassifier()
    rf_model_chars = rf_chars.fit(x_train_tf_idf_chars, train_y)
    accuracy_chars = cross_val_score(rf_model_chars, x_test_tf_idf_chars, test_y, cv=10).mean()
    print("Accuracy TF-IDF Characters: %.3f" % accuracy_chars)

    return rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars

rf_model_count, rf_model_word, rf_model_ngram, rf_model_chars = create_model_randomforest(train_x, test_x)

def predict_sentiment(model, vectorizer, new_comment):
    new_comment_series = pd.Series(new_comment)
    new_comment_transformed = vectorizer.transform(new_comment_series)
    result = model.predict(new_comment_transformed)
    if result == 1:
        print("Comment is Positive")
    else:
        print("Comment is Negative")
        
# Assuming you have already trained the models and vectorizers
# Save each vectorizer and model separately
with open('count_vectorizer.pkl', 'wb') as f:
    pickle.dump(count_vectorizer, f)
    
with open('rf_model_count.pkl', 'wb') as f:
    pickle.dump(rf_model_count, f)

with open('tfidf_word_vectorizer.pkl', 'wb') as f:
    pickle.dump(tf_idf_word_vectorizer, f)
with open('rf_model_word.pkl', 'wb') as f:
    pickle.dump(rf_model_word, f)

with open('tfidf_ngram_vectorizer.pkl', 'wb') as f:
    pickle.dump(tf_idf_ngram_vectorizer, f)
with open('rf_model_ngram.pkl', 'wb') as f:
    pickle.dump(rf_model_ngram, f)

with open('tfidf_chars_vectorizer.pkl', 'wb') as f:
    pickle.dump(tf_idf_chars_vectorizer, f)
with open('rf_model_chars.pkl', 'wb') as f:
    pickle.dump(rf_model_chars, f)

[nltk_data] Downloading package stopwords to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Dawood
[nltk_data]     MD\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Accuracy - Count Vectors: 0.883
Accuracy - TF-IDF Word: 0.867
Accuracy TF-IDF ngram: 0.867
Accuracy TF-IDF Characters: 0.883
